In [1]:
import json
import pandas
import pathlib
import pydash
import requests
import time
import tqdm

data_path = pathlib.Path.cwd() / 'data.parquet'

if not data_path.exists():

    index = requests.get('https://www.bfi.org.uk/sight-and-sound/greatest-films-all-time/all-voters').text
    index = index.split('<script type="text/javascript">var initialPageState = ')[1].split('</script>')[0]
    dataframe = pandas.DataFrame(pydash.get(json.loads(index), 'componentState.allVoters'))
    dataframe = dataframe[['firstname', 'surname', 'type', 'country', 'url']]

    votes = pandas.DataFrame()

    for x in tqdm.tqdm(dataframe.url.unique()):

        time.sleep(4)

        vote_page = pandas.read_html('https://www.bfi.org.uk'+x, encoding='utf8')[0]
        vote_page['url'] = x
        if len(vote_page) != 10:
            print(pathlib.Path(x).stem, 'should be only ten votes')
        votes = pandas.concat([votes, vote_page])

    dataframe = pandas.merge(dataframe, votes, on='url', how='left')
    dataframe = dataframe.astype(str)
    dataframe.to_parquet(data_path)
else:
    dataframe = pandas.read_parquet(data_path)

print(len(dataframe))
dataframe.head(10)

21113


,firstname,surname,type,country,url,Film,Year,Director
0,Arun,A.K.,critic,India,/sight-and-sound/greatest-films-all-time/all-v...,GULAAL,2009,Anurag Kashyap
1,Arun,A.K.,critic,India,/sight-and-sound/greatest-films-all-time/all-v...,Gangs of Wasseypur,2012,Anurag Kashyap
2,Arun,A.K.,critic,India,/sight-and-sound/greatest-films-all-time/all-v...,ANANTARAM,1987,Adoor Gopalakrishnan
3,Arun,A.K.,critic,India,/sight-and-sound/greatest-films-all-time/all-v...,VIDHEYAN,1993,Adoor Gopalakrishnan
4,Arun,A.K.,critic,India,/sight-and-sound/greatest-films-all-time/all-v...,TRIKAAL,1986,Shyam Benegal
5,Arun,A.K.,critic,India,/sight-and-sound/greatest-films-all-time/all-v...,Lost in Translation,2003,Sofia Coppola
6,Arun,A.K.,critic,India,/sight-and-sound/greatest-films-all-time/all-v...,There Will Be Blood,2007,Paul Thomas Anderson
7,Arun,A.K.,critic,India,/sight-and-sound/greatest-films-all-time/all-v...,Taxi Driver,1976,Martin Scorsese
8,Arun,A.K.,critic,India,/sight-and-sound/greatest-films-all-time/all-v...,8½,1963,Federico Fellini
9,Arun,A.K.,critic,India,/sight-and-sound/greatest-films-all-time/all-v...,A Clockwork Orange,1971,Stanley Kubrick
